In [1]:
# make a dir if it doesn't already exsist
import os
directory = "instacart_csv"
if not os.path.exists(directory):
    os.makedirs(directory)
os.chdir(directory)


In [2]:
import boto3
import botocore

BUCKET_NAME = 'instacart-techfield' # replace with your bucket name
A = 'aisles.csv' # replace with your object key
B = "departments.csv"
C = 'order_products__prior.csv'
D = 'order_products__train.csv'
E = 'orders.csv'
F = 'products.csv'
G = 'sample_submission.csv'
s3 = boto3.resource('s3')

try:
    s3.Bucket(BUCKET_NAME).download_file(A, 'aisles.csv')
    s3.Bucket(BUCKET_NAME).download_file(B, "departments.csv")
    s3.Bucket(BUCKET_NAME).download_file(C, 'order_products__prior.csv')
    s3.Bucket(BUCKET_NAME).download_file(D, 'order_products__train.csv')
    s3.Bucket(BUCKET_NAME).download_file(E, 'orders.csv')
    s3.Bucket(BUCKET_NAME).download_file(F, 'products.csv')
    s3.Bucket(BUCKET_NAME).download_file(G, 'sample_submission.csv')
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [3]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

os.chdir("/home/ubuntu/")

spark = SparkSession.builder \
    .appName ("Convert into Parquet") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
      
a = spark.read.load("/home/ubuntu/instacart_csv/aisles.csv", format="csv", header=True)
a.write.save("instacart_parquet/aisles.parquet", format="parquet")

b = spark.read.load("/home/ubuntu/instacart_csv/departments.csv", format="csv", header=True)
b.write.save("instacart_parquet/departments.parquet", format="parquet")

c = spark.read.load("/home/ubuntu/instacart_csv/order_products__prior.csv", format="csv", header=True)
c.write.save("instacart_parquet/order_products__prior.parquet", format="parquet")

d = spark.read.load("/home/ubuntu/instacart_csv/order_products__train.csv", format="csv", header=True)
d.write.save("instacart_parquet/order_products__train.parquet", format="parquet")

e = spark.read.load("/home/ubuntu/instacart_csv/orders.csv", format="csv", header=True)
e.write.save("instacart_parquet/orders.parquet", format="parquet")

f = spark.read.load("/home/ubuntu/instacart_csv/products.csv", format="csv", header=True)
f.write.save("instacart_parquet/products.parquet", format="parquet")

g = spark.read.load("/home/ubuntu/instacart_csv/sample_submission.csv", format="csv", header=True)
g.write.save("instacart_parquet/sample_submission.parquet", format="parquet")
    


In [4]:
a = spark.read.load("/home/ubuntu/instacart_parquet/aisles.parquet/*.parquet", format="parquet")
b = spark.read.load("/home/ubuntu/instacart_parquet/departments.parquet/*.parquet", format="parquet")
c = spark.read.load("/home/ubuntu/instacart_parquet/order_products__prior.parquet/*.parquet", format="parquet")
d = spark.read.load("/home/ubuntu/instacart_parquet/order_products__train.parquet/*.parquet", format="parquet")
e = spark.read.load("/home/ubuntu/instacart_parquet/orders.parquet/*.parquet", format="parquet")
f = spark.read.load("/home/ubuntu/instacart_parquet/products.parquet/*.parquet", format="parquet")
g = spark.read.load("/home/ubuntu/instacart_parquet/sample_submission.parquet/*.parquet", format="parquet")



In [5]:
import os, fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

aisles_path = find('*.parquet', '/home/ubuntu/instacart_parquet/aisles.parquet')[0]
departments_path = find('*.parquet', '/home/ubuntu/instacart_parquet/departments.parquet')[0]
order_products__prior_path = find('*.parquet', '/home/ubuntu/instacart_parquet/order_products__prior.parquet')[0]
order_products__train_path = find('*.parquet', '/home/ubuntu/instacart_parquet/order_products__train.parquet')[0]
orders_path = find('*.parquet', '/home/ubuntu/instacart_parquet/orders.parquet')[0]
products_path = find('*.parquet', '/home/ubuntu/instacart_parquet/products.parquet')[0]
sample_submission_path = find('*.parquet', '/home/ubuntu/instacart_parquet/aisles.parquet')[0]

In [6]:
import boto3
s3_client = boto3.client('s3')
s3_client.upload_file(aisles_path, 'instacart-techfield', 'instacart_parquet/aisles.parquet')
s3_client.upload_file(departments_path, 'instacart-techfield', 'instacart_parquet/departments.parquet')
s3_client.upload_file(order_products__prior_path, 'instacart-techfield', 'instacart_parquet/order_products__prior.parquet')
s3_client.upload_file(order_products__train_path, 'instacart-techfield', 'instacart_parquet/order_products__train.parquet')
s3_client.upload_file(orders_path, 'instacart-techfield', 'instacart_parquet/orders.parquet')
s3_client.upload_file(products_path, 'instacart-techfield', 'instacart_parquet/products.parquet')
s3_client.upload_file(sample_submission_path, 'instacart-techfield', 'instacart_parquet/sample_submission.parquet')